In [19]:
import numpy as np
import math
import random as rnd

import cv2

6. Система координат камеры получается из мировой поворотом на <code>45 градусов вокруг оси z и трансляцией на 10 вдоль оси z</code>. Внутренние параметры камеры: <code>fx=fy=400, cx=960, cy=540</code>. Написать программу, которая выводит на экран матрицу проекции камеры и целочисленные координаты пикселя на изображении, соответствующего трехмерной точке с координатами <code>(10, -10, 100)</code> в мировой системе координат.

In [20]:
r = np.around(math.cos(math.radians(45)),3)

rt = np.array([[math.cos(math.radians(45)), -math.sin(math.radians(45)), 0, 0],
              [math.sin(math.radians(45)), math.cos(math.radians(45)), 0, 0],
              [0, 0, 1, 10]], dtype=np.float64)

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]], dtype = np.float64)

point = np.array([10, -10, 100, 1], dtype = np.float64)

p = k @ rt

n_point = p @ point

p = np.around(p, 2)

n_point = np.around(n_point)

print("Матрица проекции камеры:\n")
p


Матрица проекции камеры:



array([[ 2.8284e+02, -2.8284e+02,  9.6000e+02,  9.6000e+03],
       [ 2.8284e+02,  2.8284e+02,  5.4000e+02,  5.4000e+03],
       [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+01]])

In [21]:
print("Координаты пикселя: \n")

print("x = " + str(n_point[0]/n_point[2]))
print("y = " + str(n_point[1]/n_point[2]))

Координаты пикселя: 

x = 1011.4272727272727
y = 540.0


7. Используя прямое линейное преобразование (<code>DLT</code>) и <code>SVD</code>, написать программу, которая принимает на вход множество пар двумерных точек, и выводит на экран соответствующую этим парам матрицу гомографии. Написать генератор пар двумерных точек для теста алгоритма, удовлетворяющих заданному заранее преобразованию гомографии.

In [22]:
def pairs_generator(n):
    H = [[4, 2, 1],
         [5, 7, 3],
         [8, 9, 4]]
    H = np.array(H)
    pairs = []
    for i in range(n):
        x1 = rnd.randint(0, 100)
        y1 = rnd.randint(0, 100)
        p2 = H @ np.array([x1, y1, 1])
        p2 = p2 / p2[2]
        pairs.append(([x1, y1, 1], p2))
    return pairs

Решение задачи гомографии приводит к решению системы однородных линейных уравнений, таких как:

\begin{equation}
\left(
\begin{array}{ccccccccc}
-x1 & -y1 & -1 & 0 & 0 & 0 & x1*xp1 & y1*xp1 & xp1\\
0 & 0 & 0 & -x1 & -y1 & -1 & x1*yp1 & y1*yp1 &  yp1\\
-x2 & -y2 & -1 & 0 & 0 & 0 & x2*xp2 & y2*xp2 & xp2\\
0 & 0 & 0 & -x2 & -y2 & -1 & x2*yp2 & y2*yp2 & yp2\\
-x3 & -y3 & -1 & 0 & 0 & 0 & x3*xp3 & y3*xp3 & xp3\\
0 & 0 & 0 & -x3 & -y3 & -1 & x3*yp3 & y3*yp3 & yp3\\
-x4 & -y4 & -1 & 0 & 0 & 0 & x4*xp4 & y4*xp4 & xp4\\
0 & 0 & 0 & -x4 & -y4 & -1 & x4*yp4 & y4*yp4 & yp4\\
\end{array}
\right) *H=0
\end{equation}

In [23]:
##This function will find the homography betweeb 4 points using svd
def homography(pairs):
    x1 = pairs[0][0][0]
    y1 = pairs[0][0][1]
    x2 = pairs[1][0][0]
    y2 = pairs[1][0][1]
    x3 = pairs[2][0][0]
    y3 = pairs[2][0][1]
    x4 = pairs[3][0][0]
    y4 = pairs[3][0][1]
    xp1 = pairs[0][1][0]
    yp1 = pairs[0][1][1]
    xp2 = pairs[1][1][0]
    yp2 = pairs[1][1][1]
    xp3 = pairs[2][1][0]
    yp3 = pairs[2][1][1]
    xp4 = pairs[3][1][0]
    yp4 = pairs[3][1][1]

    matrix=[
	[-x1, -y1, -1, 0, 0, 0, x1*xp1, y1*xp1, xp1],
	[0, 0, 0, -x1, -y1, -1, x1*yp1, y1*yp1, yp1],
	[-x2, -y2, -1, 0, 0, 0, x2*xp2, y2*xp2, xp2],
	[0, 0, 0, -x2, -y2, -1, x2*yp2, y2*yp2, yp2],
	[-x3, -y3, -1, 0, 0, 0, x3*xp3, y3*xp3, xp3],
	[0, 0, 0, -x3, -y3, -1, x3*yp3, y3*yp3, yp3],
	[-x4, -y4, -1, 0, 0, 0, x4*xp4, y4*xp4, xp4],
	[0, 0, 0, -x4, -y4, -1, x4*yp4, y4*yp4, yp4]]
    U, S, Vt = np.linalg.svd(matrix)
    V = Vt.transpose()[:, len(Vt)-1]
    H = V.reshape(3,3)
    return H

In [24]:
pairs = pairs_generator(6)
H = homography(pairs)
H

array([[ 3.90360029e-01,  1.95180014e-01,  9.75900102e-02],
       [ 4.87950036e-01,  6.83130050e-01,  2.92770026e-01],
       [ 1.04222503e-14, -2.42294360e-13,  9.75900071e-02]])

In [25]:
p2 = H @ pairs[0][0]
p2 = p2 / p2[2]
p2

array([225., 436.,   1.])

In [26]:
p1 = pairs[0][1] / pairs[0][1][2]
p1

array([225, 436, 622])

In [27]:
h_cv2, mask = cv2.findHomography(pairs[:, 0], pairs[:, 1])
h_cv2

TypeError: list indices must be integers or slices, not tuple

8. Модифицировать генератор из <code>Задачи 7</code> так, чтобы он выдавал <code>n=1000</code> соответствий, из которых <code>30%</code> соответствуют заданному заранее преобразованию гомографии, а остальные взяты случайно (преобразованию гомографии не удовлетворяют). Реализовать нахождение преобразования гомографии, используя <code>DLT</code> и <code>RANSAC</code>.